# IBM Applied Data Science Capstone Project: The Battle of Seattle Neighborhoods

## Table of Contents
> Introduction<br>
> Data and Sources<br>
> Methodology<br>
> Analysis<br>
> Results and Discussion<br>
> Conclusion

## Introduction

Graduating from college is a big deal! My parents are just as excited as I am, and they were planning to fly to Seattle to attend my graduation. Unfortunately, the plan is cancelled due to the COVID-19 outbreak… But if they were to come here in Seattle to attend my college graduation, where would be the best location for them to stay? The most ideal location should be close to local attractions, nearby restaurants, public transportation, and more importantly, it can’t be too far from my school! In this project, I will be using what we learned in this course to try to find this ideal location. 

## Data and Sources

I will need to search for or generate the following data to solve the problem:
> •	Coordinates of my school (University of Washington) using Google Maps API geocoding<br>
•	Centers of candidate areas and their approximate addresses using Google Maps API geocoding<br>
•	Number of restaurants and their ratings in every candidate neighborhood using Foursquare API<br>
•	Number of attractions in every candidate neighborhood using Foursquare API

## Methodology

In this project we will mainly search for areas of Seattle that have high restaurant density and high attraction density. We will limit our analysis to areas about 6 miles around university of Washington.<br><br>
> In the first step, we have collected coordinate data on my school, University of Washington, using Google Maps API geocoding. Then, we limited our search area to within the radius 6 miles from my school.<br><br>
In the next step, we collected data on restaurant densities on those qualifying areas, areas within 6 miles radius from my school (using heatmaps).<br><br>
In the final step, we presented a map of all such locations as well as clusters (using k-means clustering) of those locations to identify general neighborhoods that could be considered as a optimal zone for my parents to stay during their visit for my graduation. 


## Analysis

### 1. Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format='retina'
import os
from sklearn.cluster import KMeans
!pip install folium
import folium 
from geopy.geocoders import Nominatim
!pip install geocoder
import geocoder
import matplotlib.cm as cm
import matplotlib.colors as colors
from bs4 import BeautifulSoup
import requests

     |████████████████████████████████| 102kB 9.2MB/s ta 0:00:011
     |████████████████████████████████| 102kB 10.9MB/s ta 0:00:01


### 2. Scrap data from Wikipedia into a DataFrame

In [2]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighborhoods_in_Seattle").text
soup = BeautifulSoup(data, 'html.parser')
neighborhoodList = []
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)
sea_df = pd.DataFrame({"Neighborhood": neighborhoodList})
sea_df.head()

,Neighborhood
0,"► Ballard, Seattle‎ (1 C, 16 P)"
1,"► Beacon Hill, Seattle‎ (10 P)"
2,"► Belltown, Seattle‎ (20 P)"
3,"► Broadview, Seattle‎ (3 P)"
4,"► Capitol Hill, Seattle‎ (46 P)"


### 3. Get the geographical coordinates

In [3]:
def get_latlng(neighborhood):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, North Seattle, United States'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [4]:
coords = [ get_latlng(neighborhood) for neighborhood in sea_df["Neighborhood"].tolist() ]

In [5]:
coords

[[47.65781250347969, -122.37623099999999],
 [47.56201049134293, -122.30514500340632],
 [47.61576000000008, -122.34463999999997],
 [47.722380000000044, -122.36497999999995],
 [47.62970002553527, -122.30861002279389],
 [47.58935738449369, -122.38816763449448],
 [47.605530000000044, -122.33431999999999],
 [39.952224, -75.141729],
 [47.59917000000007, -122.32795999999996],
 [47.59680000000003, -122.33423000779023],
 [47.58935738449369, -122.38816763449448],
 [47.618157153503944, -122.32918095979659],
 [47.60878000000008, -122.32642999999996],
 [47.65114997446483, -122.34994004443398],
 [47.547510000000045, -122.32148999999998],
 [47.633480999999996, -122.38702840990703],
 [47.647309496355795, -122.3046128415523],
 [47.59868000000006, -122.33290999999997],
 [47.64254081238128, -122.351152205849],
 [47.512350000000026, -122.26276999999999],
 [47.55123000000003, -122.28674999999998],
 [47.58935738449369, -122.38816763449448],
 [47.61853000000008, -122.34328999999997],
 [47.66127000000006, -12

In [6]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [7]:
sea_df['Latitude'] = df_coords['Latitude']
sea_df['Longitude'] = df_coords['Longitude']
print(sea_df.shape)
sea_df

(27, 3)


,Neighborhood,Latitude,Longitude
0,"► Ballard, Seattle‎ (1 C, 16 P)",47.657813,-122.376231
1,"► Beacon Hill, Seattle‎ (10 P)",47.562010,-122.305145
2,"► Belltown, Seattle‎ (20 P)",47.615760,-122.344640
3,"► Broadview, Seattle‎ (3 P)",47.722380,-122.364980
4,"► Capitol Hill, Seattle‎ (46 P)",47.629700,-122.308610
5,"► Cascade, Seattle‎ (4 P)",47.589357,-122.388168
6,"► Central District, Seattle‎ (12 P)",47.605530,-122.334320
7,"► Central Waterfront, Seattle‎ (13 P)",39.952224,-75.141729
8,"► Chinatown-International District, Seattle‎ ...",47.599170,-122.327960
9,"► Denny Triangle, Seattle‎ (13 P)",47.596800,-122.334230


In [8]:
sea_df.to_csv("sea_df.csv", index=False)

### 4. Create a map of Seattle with neighborhoods names

In [9]:
address = 'North Seattle, United States'
geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North Seattle, United States {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North Seattle, United States 47.6607729, -122.29149712901432.


In [10]:
map_sea = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(sea_df['Latitude'], sea_df['Longitude'], sea_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_sea)  
    
map_sea

In [11]:
map_sea.save('map_sea.html')

### 5. Use Foursquare API to explore the neighborhoods

In [12]:
CLIENT_ID = '1CNG4RCXWNTCNCHN52TDMFBEKNB5Z10V11E33NTEHEH4XNJ5' # your Foursquare ID
CLIENT_SECRET = '2AMW4ZP5VUBX0GSUTJA3B5PP1ELPJ4031HI244MK2EMNRTOE' # your Foursquare Secret
VERSION = '20200515' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1CNG4RCXWNTCNCHN52TDMFBEKNB5Z10V11E33NTEHEH4XNJ5
CLIENT_SECRET:2AMW4ZP5VUBX0GSUTJA3B5PP1ELPJ4031HI244MK2EMNRTOE


In [13]:
radius = 6000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(sea_df['Latitude'], sea_df['Longitude'], sea_df['Neighborhood']):
    
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()['response']['groups'][0]['items']
    
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [14]:
venues_df = pd.DataFrame(venues)

venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(2700, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,"► Ballard, Seattle‎ (1 C, 16 P)",47.657813,-122.376231,Rooftop Brewing Company,47.655766,-122.373127,Brewery
1,"► Ballard, Seattle‎ (1 C, 16 P)",47.657813,-122.376231,Trader Joe's,47.662783,-122.374427,Grocery Store
2,"► Ballard, Seattle‎ (1 C, 16 P)",47.657813,-122.376231,Wild Salmon Seafood Market,47.655326,-122.380985,Fish Market
3,"► Ballard, Seattle‎ (1 C, 16 P)",47.657813,-122.376231,Reuben's Brews,47.665398,-122.373270,Brewery
4,"► Ballard, Seattle‎ (1 C, 16 P)",47.657813,-122.376231,Stoup Brewing,47.666551,-122.371277,Brewery


In [15]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
"► Ballard, Seattle‎ (1 C, 16 P)",100,100,100,100,100,100
"► Beacon Hill, Seattle‎ (10 P)",100,100,100,100,100,100
"► Belltown, Seattle‎ (20 P)",100,100,100,100,100,100
"► Broadview, Seattle‎ (3 P)",100,100,100,100,100,100
"► Capitol Hill, Seattle‎ (46 P)",100,100,100,100,100,100
"► Cascade, Seattle‎ (4 P)",100,100,100,100,100,100
"► Central District, Seattle‎ (12 P)",100,100,100,100,100,100
"► Central Waterfront, Seattle‎ (13 P)",100,100,100,100,100,100
"► Chinatown-International District, Seattle‎ (11 P)",100,100,100,100,100,100


In [16]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 181 uniques categories.


In [17]:
venues_df['VenueCategory'].unique()[:50]

array(['Brewery', 'Grocery Store', 'Fish Market', 'Farmers Market',
       'Donut Shop', 'Mexican Restaurant', 'Ice Cream Shop', 'Beer Bar',
       'Gym', 'Mediterranean Restaurant', 'Pizza Place', 'Post Office',
       'Park', 'Coffee Shop', 'Bakery', 'Supermarket', 'Chocolate Shop',
       'Canal Lock', 'Arcade', 'Winery', 'Japanese Restaurant',
       'Bookstore', 'Seafood Restaurant', 'Bar', 'French Restaurant',
       'Playground', 'Wine Shop', 'Zoo', 'Burger Joint', 'Pie Shop',
       'Trail', 'Organic Grocery', 'Wine Bar', 'Italian Restaurant',
       'Caribbean Restaurant', 'Outdoor Sculpture', 'Chinese Restaurant',
       'American Restaurant', 'Deli / Bodega', 'Café',
       'New American Restaurant', 'Cupcake Shop', 'Scenic Lookout',
       'Poke Place', 'Pet Store', 'Greek Restaurant', 'Beer Store',
       'Fried Chicken Joint', 'Boat or Ferry', 'Beach'], dtype=object)

In [18]:
"Restaurant" in venues_df['VenueCategory'].unique()

True

### 6. Analyze Each Neighborhood

In [19]:
sea_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

sea_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

fixed_columns = [sea_onehot.columns[-1]] + list(sea_onehot.columns[:-1])
sea_onehot = sea_onehot[fixed_columns]

print(sea_onehot.shape)
sea_onehot.head()

(2700, 182)


,Neighborhoods,African Restaurant,Airport,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Video Game Store,Video Store,Vietnamese Restaurant,Waterfront,Wine Bar,Wine Shop,Winery,Wings Joint,Yoga Studio,Zoo
0,"► Ballard, Seattle‎ (1 C, 16 P)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"► Ballard, Seattle‎ (1 C, 16 P)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"► Ballard, Seattle‎ (1 C, 16 P)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"► Ballard, Seattle‎ (1 C, 16 P)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"► Ballard, Seattle‎ (1 C, 16 P)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
sea_grouped = sea_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(sea_grouped.shape)
sea_grouped

(27, 182)


,Neighborhoods,African Restaurant,Airport,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Video Game Store,Video Store,Vietnamese Restaurant,Waterfront,Wine Bar,Wine Shop,Winery,Wings Joint,Yoga Studio,Zoo
0,"► Ballard, Seattle‎ (1 C, 16 P)",0.00,0.00,0.02,0.00,0.01,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.01,0.01,0.01,0.00,0.00,0.01
1,"► Beacon Hill, Seattle‎ (10 P)",0.01,0.01,0.00,0.01,0.00,0.00,0.00,0.01,0.01,...,0.00,0.00,0.01,0.00,0.00,0.01,0.00,0.00,0.00,0.00
2,"► Belltown, Seattle‎ (20 P)",0.00,0.00,0.01,0.00,0.00,0.00,0.01,0.00,0.00,...,0.00,0.00,0.00,0.01,0.01,0.01,0.00,0.00,0.02,0.00
3,"► Broadview, Seattle‎ (3 P)",0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.01,...,0.00,0.00,0.01,0.00,0.00,0.01,0.00,0.00,0.00,0.00
4,"► Capitol Hill, Seattle‎ (46 P)",0.00,0.00,0.01,0.00,0.00,0.01,0.00,0.00,0.00,...,0.00,0.00,0.01,0.00,0.02,0.00,0.00,0.00,0.03,0.00
5,"► Cascade, Seattle‎ (4 P)",0.00,0.00,0.01,0.00,0.00,0.00,0.02,0.00,0.01,...,0.00,0.00,0.01,0.01,0.00,0.01,0.00,0.00,0.01,0.00
6,"► Central District, Seattle‎ (12 P)",0.00,0.00,0.01,0.00,0.00,0.01,0.01,0.00,0.00,...,0.00,0.00,0.05,0.01,0.01,0.01,0.00,0.00,0.02,0.00
7,"► Central Waterfront, Seattle‎ (13 P)",0.00,0.00,0.01,0.00,0.00,0.00,0.01,0.00,0.00,...,0.00,0.00,0.00,0.01,0.01,0.00,0.00,0.00,0.00,0.00
8,"► Chinatown-International District, Seattle‎ ...",0.00,0.00,0.01,0.00,0.00,0.01,0.01,0.00,0.00,...,0.00,0.00,0.05,0.01,0.01,0.01,0.00,0.00,0.02,0.00
9,"► Denny Triangle, Seattle‎ (13 P)",0.00,0.00,0.01,0.00,0.00,0.01,0.01,0.00,0.00,...,0.00,0.00,0.05,0.01,0.01,0.01,0.00,0.00,0.02,0.00


In [21]:
len(sea_grouped[sea_grouped["Restaurant"] > 0])

19

In [22]:
sea_restaurant = sea_grouped[["Neighborhoods","Restaurant"]]

In [23]:
sea_restaurant.head()

,Neighborhoods,Restaurant
0,"► Ballard, Seattle‎ (1 C, 16 P)",0.00
1,"► Beacon Hill, Seattle‎ (10 P)",0.01
2,"► Belltown, Seattle‎ (20 P)",0.01
3,"► Broadview, Seattle‎ (3 P)",0.00
4,"► Capitol Hill, Seattle‎ (46 P)",0.01


### 7. Clustering Neighborhoods

In [24]:
kclusters = 3

sea_clustering = sea_restaurant.drop(["Neighborhoods"], 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(sea_clustering)

kmeans.labels_[0:10]

array([1, 0, 0, 1, 0, 0, 0, 1, 0, 0], dtype=int32)

In [25]:
sea_merged = sea_restaurant.copy()
sea_merged["Cluster Labels"] = kmeans.labels_

In [26]:
sea_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
sea_merged.head()

,Neighborhood,Restaurant,Cluster Labels
0,"► Ballard, Seattle‎ (1 C, 16 P)",0.00,1
1,"► Beacon Hill, Seattle‎ (10 P)",0.01,0
2,"► Belltown, Seattle‎ (20 P)",0.01,0
3,"► Broadview, Seattle‎ (3 P)",0.00,1
4,"► Capitol Hill, Seattle‎ (46 P)",0.01,0


In [27]:
sea_merged = sea_merged.join(sea_df.set_index("Neighborhood"), on="Neighborhood")
print(sea_merged.shape)
sea_merged.head()

(27, 5)


,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
0,"► Ballard, Seattle‎ (1 C, 16 P)",0.00,1,47.657813,-122.376231
1,"► Beacon Hill, Seattle‎ (10 P)",0.01,0,47.562010,-122.305145
2,"► Belltown, Seattle‎ (20 P)",0.01,0,47.615760,-122.344640
3,"► Broadview, Seattle‎ (3 P)",0.00,1,47.722380,-122.364980
4,"► Capitol Hill, Seattle‎ (46 P)",0.01,0,47.629700,-122.308610


In [28]:
print(sea_merged.shape)
sea_merged.sort_values(["Cluster Labels"], inplace=True)
sea_merged

(27, 5)


,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
26,"► West Seattle, Seattle‎ (36 P)",0.01,0,47.607620,-122.333590
17,"► Pioneer Square, Seattle‎ (20 P)",0.01,0,47.598680,-122.332910
21,"► SoDo, Seattle‎ (12 P)",0.01,0,47.589357,-122.388168
15,"► Magnolia, Seattle‎ (13 P)",0.01,0,47.633481,-122.387028
14,"► Georgetown, Seattle‎ (6 P)",0.01,0,47.547510,-122.321490
22,"► South Lake Union, Seattle‎ (22 P)",0.01,0,47.618530,-122.343290
12,"► First Hill, Seattle‎ (1 C, 10 P)",0.01,0,47.608780,-122.326430
11,"► Eastlake, Seattle‎ (6 P)",0.01,0,47.618157,-122.329181
10,"► Downtown Seattle‎ (3 C, 64 P)",0.01,0,47.589357,-122.388168
9,"► Denny Triangle, Seattle‎ (13 P)",0.01,0,47.596800,-122.334230


In [34]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(sea_merged['Latitude'], sea_merged['Longitude'], sea_merged['Neighborhood'], sea_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [30]:
map_clusters.save('map_clusters.html')

### 8. Examine Clusters

In [31]:
sea_merged.loc[sea_merged['Cluster Labels'] == 0]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
26,"► West Seattle, Seattle‎ (36 P)",0.01,0,47.607620,-122.333590
17,"► Pioneer Square, Seattle‎ (20 P)",0.01,0,47.598680,-122.332910
21,"► SoDo, Seattle‎ (12 P)",0.01,0,47.589357,-122.388168
15,"► Magnolia, Seattle‎ (13 P)",0.01,0,47.633481,-122.387028
14,"► Georgetown, Seattle‎ (6 P)",0.01,0,47.547510,-122.321490
22,"► South Lake Union, Seattle‎ (22 P)",0.01,0,47.618530,-122.343290
12,"► First Hill, Seattle‎ (1 C, 10 P)",0.01,0,47.608780,-122.326430
11,"► Eastlake, Seattle‎ (6 P)",0.01,0,47.618157,-122.329181
10,"► Downtown Seattle‎ (3 C, 64 P)",0.01,0,47.589357,-122.388168
9,"► Denny Triangle, Seattle‎ (13 P)",0.01,0,47.596800,-122.334230


In [32]:
sea_merged.loc[sea_merged['Cluster Labels'] == 1]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
23,"► University District, Seattle‎ (1 C, 15 P)",0.0,1,47.661270,-122.313070
24,"► Wallingford, Seattle‎ (5 P)",0.0,1,47.693001,-122.336484
0,"► Ballard, Seattle‎ (1 C, 16 P)",0.0,1,47.657813,-122.376231
16,"► Montlake, Seattle‎ (9 P)",0.0,1,47.647309,-122.304613
25,"► Wedgwood, Seattle‎ (5 P)",0.0,1,47.687010,-122.294940
7,"► Central Waterfront, Seattle‎ (13 P)",0.0,1,39.952224,-75.141729
3,"► Broadview, Seattle‎ (3 P)",0.0,1,47.722380,-122.364980
13,"► Fremont, Seattle‎ (15 P, 1 F)",0.0,1,47.651150,-122.349940


In [33]:
sea_merged.loc[sea_merged['Cluster Labels'] == 2]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
20,"► Rainier Valley, Seattle‎ (1 C, 9 P)",0.02,2,47.55123,-122.28675


## Results and Discussion

Most of the restaurants are concentrated in rainier valley, with the highest number in cluster 2 and moderate number in cluster 1. On the other hand, cluster 0 has a relatively small number of restaurants in the neighborhoods. This means that there are not a lot of choices for restaurants in that neighborhood and this area might not be the most ideal location for my parents to stay. Meanwhile, number of restaurants in cluster 2 ranks top in all neighborhoods and might be a great choice to stay. Therefore, this project indicates that if my parents were here to attend graduation and were looking for somewhere with a lot of restaurants to stay, cluster 2 might be their best choice, where cluster 0 will be the next option and cluster 1 for the last option.

## Conclusion

In this project, through thorough analysis with the help from various sources, we can conclude that if my parents were here to attend graduation and were looking for somewhere with a lot of restaurants to stay, rainier valley would be the best neighborhood for them to stay.